In [1]:
import requests 
import pandas as pd
import numpy as np
import time

### Get box score for each player for each season since 2000-01

In [60]:
from nba_api.stats.endpoints import playergamelogs

season_names = ['2000-01','2001-02','2002-03','2003-04','2004-05','2005-06','2006-07','2007-08',\
                '2008-09','2009-10','2010-11','2011-12','2012-13','2013-14','2014-15','2015-16',\
                '2016-17','2017-18','2018-19','2019-20','2020-21']

season_game_logs_list = []

for name in season_names:
    print(name)
    seasongamelogs = playergamelogs.PlayerGameLogs(season_nullable = name)
    df = seasongamelogs.player_game_logs.get_data_frame()
    season_game_logs_list.append(df)
    time.sleep(1)

2000-01
2001-02
2002-03
2003-04
2004-05
2005-06
2006-07
2007-08
2008-09
2009-10
2010-11
2011-12
2012-13
2013-14
2014-15
2015-16
2016-17
2017-18
2018-19
2019-20
2020-21


In [62]:
check_rows = 0

for df in season_game_logs_list:
    check_rows += len(df)
    
print(check_rows)

season_game_logs_df = pd.concat(season_game_logs_list)
print(len(season_game_logs_df))

518141

In [66]:
season_game_logs_df = season_game_logs_df.reset_index().iloc[:,1:]
season_game_logs_df.to_csv('2000-2021 Season Game Logs.csv')

In [3]:
season_game_logs = pd.read_csv('2000-2021 Season Game Logs.csv',index_col=0,dtype={'GAME_ID':'str'})
season_game_logs.tail()

,SEASON_YEAR,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,...,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,VIDEO_AVAILABLE_FLAG
518136,2020-21,101150,Lou Williams,Lou,1610612746,LAC,LA Clippers,0022000002,2020-12-22T00:00:00,LAC @ LAL,...,7210,1,10833,3793,8773,9911,13031,1849,143,1
518137,2020-21,1629662,Mfiondu Kabengele,Mfiondu,1610612746,LAC,LA Clippers,0022000002,2020-12-22T00:00:00,LAC @ LAL,...,7210,1,1,16020,20448,12607,22167,1849,143,1
518138,2020-21,1630164,James Wiseman,James,1610612744,GSW,Golden State Warriors,0022000001,2020-12-22T00:00:00,GSW @ BKN,...,7210,1,10833,2281,3538,18716,5345,1849,143,1
518139,2020-21,1629308,Juan Toscano-Anderson,Juan,1610612744,GSW,Golden State Warriors,0022000001,2020-12-22T00:00:00,GSW @ BKN,...,697,1,16132,10308,16353,18135,12463,1849,143,1
518140,2020-21,1628379,Luke Kennard,Luke,1610612746,LAC,LA Clippers,0022000002,2020-12-22T00:00:00,LAC @ LAL,...,2271,1,16132,16020,16353,13536,12706,1849,143,1


### Get advanced box score for each player for each season since 2000-01

In [1]:
from nba_api.stats.endpoints import boxscoreadvancedv2

boxscoreadvanced = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id ='0020000106',start_period=1,end_period=1,start_range=0,end_range=0,range_type=0)
df = boxscoreadvanced.player_stats.get_data_frame()
df.head()

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,...,TM_TOV_PCT,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,POSS,PIE
0,0020000106,1610612741,CHI,Chicago,1897,Metta World Peace,Metta,F,,32:39,...,11.8,0.455,0.505,0.192,0.196,90.15,90.41,75.34,62,0.085
1,0020000106,1610612741,CHI,Chicago,1882,Elton Brand,Elton,F,,27:39,...,27.3,0.500,0.538,0.328,0.331,90.55,92.01,76.67,53,0.066
2,0020000106,1610612741,CHI,Chicago,1913,Michael Ruffin,Michael,C,,25:28,...,0.0,1.000,0.868,0.054,0.053,87.90,89.56,74.63,46,0.063
3,0020000106,1610612741,CHI,Chicago,2064,Khalid El-Amin,Khalid,G,,24:32,...,0.0,0.357,0.421,0.143,0.143,90.12,90.98,75.82,46,0.138
4,0020000106,1610612741,CHI,Chicago,1500,Ron Mercer,Ron,G,,35:50,...,15.8,0.267,0.283,0.224,0.226,88.84,89.75,74.79,67,-0.005


In [4]:
all_game_ids = season_game_logs['GAME_ID'].unique()
len(all_game_ids)

25104

In [5]:
game_ids_1 = all_game_ids[0:8368]
game_ids_2 = all_game_ids[8368:16736]
game_ids_3 = all_game_ids[16736:25104]

In [11]:
# Part 1

advanced_box_scores_1_list = []

def get_adv_box_for_game(game_id,start_period=1,end_period=1,start_range=0,end_range=0,range_type=0):
    boxscoreadvanced = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id=game_id,start_period=start_period,end_period=end_period,start_range=start_range,end_range=end_range,range_type=range_type)
    df = boxscoreadvanced.player_stats.get_data_frame()
    return df
    
for game_id in game_ids_1:
    try:
        df = get_adv_box_for_game(game_id=game_id,start_period=1,end_period=1,start_range=0,end_range=0,range_type=0)
        advanced_box_scores_1_list.append(df)
        time.sleep(0.6)
        if len(advanced_box_scores_1_list) % 250 == 0:
            print(len(advanced_box_scores_1_list))
    except Exception as e:
        print("ID Errored At:",game_id)
        print(e)
        for i in range(5):
            try:
                print("trying again")
                time.sleep(30+(i*10))
                df = get_adv_box_for_game(game_id=game_id,start_period=1,end_period=1,start_range=0,end_range=0,range_type=0)
                advanced_box_scores_1_list.append(df)
            except:
                continue
            break


250
500
750
1000
1250
1500
1750
2000
2250
2500
2750
3000
3250
3500
3750
4000
ID Errored At: 0020300740
Expecting ',' delimiter: line 1 column 5073 (char 5072)
trying again
4250
ID Errored At: 0020300426
HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
trying again
4500
4750
5000
5250
5500
5750
6000
6250
6500
6750
7000
7250
7500
7750
8000
8250


In [13]:
advanced_box_scores_1_df = pd.concat(advanced_box_scores_1_list)
advanced_box_scores_1_df.to_csv('Advanced Box Scores 1.csv')

In [15]:
# Part 2

advanced_box_scores_2_list = []
    
for game_id in game_ids_2:
    try:
        df = get_adv_box_for_game(game_id=game_id,start_period=1,end_period=1,start_range=0,end_range=0,range_type=0)
        advanced_box_scores_2_list.append(df)
        time.sleep(0.6)
        if len(advanced_box_scores_2_list) % 250 == 0:
            print(len(advanced_box_scores_2_list))
    except Exception as e:
        print("ID Errored At:",game_id)
        print(e)
        for i in range(5):
            try:
                print("trying again")
                time.sleep(30+(i*10))
                df = get_adv_box_for_game(game_id=game_id,start_period=1,end_period=1,start_range=0,end_range=0,range_type=0)
                advanced_box_scores_2_list.append(df)
            except:
                continue
            break


250
500
750
1000
1250
1500
1750
2000
2250
2500
2750
3000
3250
3500
3750
4000
4250
4500
4750
5000
5250
5500
5750
6000
6250
6500
6750
7000
ID Errored At: 0021200082
HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
trying again
7250
7500
7750
8000
8250


In [16]:
advanced_box_scores_2_df = pd.concat(advanced_box_scores_2_list)
advanced_box_scores_2_df.to_csv('Advanced Box Scores 2.csv')

In [17]:
# Part 3

advanced_box_scores_3_list = []
    
for game_id in game_ids_3:
    try:
        df = get_adv_box_for_game(game_id=game_id,start_period=1,end_period=1,start_range=0,end_range=0,range_type=0)
        advanced_box_scores_3_list.append(df)
        time.sleep(0.6)
        if len(advanced_box_scores_3_list) % 250 == 0:
            print(len(advanced_box_scores_3_list))
    except Exception as e:
        print("ID Errored At:",game_id)
        print(e)
        for i in range(5):
            try:
                print("trying again")
                time.sleep(30+(i*10))
                df = get_adv_box_for_game(game_id=game_id,start_period=1,end_period=1,start_range=0,end_range=0,range_type=0)
                advanced_box_scores_3_list.append(df)
            except:
                continue
            break


250
500
750
1000
1250
1500
1750
2000
2250
ID Errored At: 0021500113
HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
trying again
2500
2750
3000
3250
3500
3750
4000
4250
4500
4750
5000
5250
5500
5750
6000
6250
6500
ID Errored At: 0021900605
Expecting value: line 1 column 1 (char 0)
trying again
ID Errored At: 0021900608
Expecting value: line 1 column 1 (char 0)
trying again
ID Errored At: 0021900542
Expecting value: line 1 column 1 (char 0)
trying again
6750
7000
ID Errored At: 0021900075
Expecting value: line 1 column 1 (char 0)
trying again
7250
7500
7750
8000
8250


In [18]:
advanced_box_scores_3_df = pd.concat(advanced_box_scores_3_list)
advanced_box_scores_3_df.to_csv('Advanced Box Scores 3.csv')

In [3]:
from nba_api.stats.endpoints import teamgamelogs

season_team_game_logs_list = []

season_names = ['2000-01','2001-02','2002-03','2003-04','2004-05','2005-06','2006-07','2007-08',\
                '2008-09','2009-10','2010-11','2011-12','2012-13','2013-14','2014-15','2015-16',\
                '2016-17','2017-18','2018-19','2019-20','2020-21']

for name in season_names:
    print(name)
    seasonteamgamelogs = teamgamelogs.TeamGameLogs(season_nullable = name)
    df = seasonteamgamelogs.team_game_logs.get_data_frame()
    season_team_game_logs_list.append(df)
    time.sleep(1)

2000-01
2001-02
2002-03
2003-04
2004-05
2005-06
2006-07
2007-08
2008-09
2009-10
2010-11
2011-12
2012-13
2013-14
2014-15
2015-16
2016-17
2017-18
2018-19
2019-20
2020-21


In [4]:
check_rows = 0

for df in season_team_game_logs_list:
    check_rows += len(df)
    
print(check_rows)

season_team_game_logs_df = pd.concat(season_team_game_logs_list)
print(len(season_team_game_logs_df))

50208
50208


In [5]:
season_team_game_logs_df = season_team_game_logs_df.reset_index().iloc[:,1:]
season_team_game_logs_df.to_csv('2000-2021 Season Team Game Logs.csv')